# lesswrong challenge: Colonizing the SuperHyperSphere

problem: https://www.lesswrong.com/posts/Rpjrwspx2QZuHbmPE/d-and-d-sci-fi-colonizing-the-superhypersphere-evaluation

submit: https://h-b-p.github.io/d-and-d-sci-SuperHyperSphere/

## setup

In [1]:
!wget https://raw.githubusercontent.com/sfgeekgit/lesswrongDataDzppg/main/cleared_sites_formated.csv
!wget https://raw.githubusercontent.com/sfgeekgit/lesswrongDataDzppg/main/measured_data.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# import data
unlabeled = pd.read_csv('cleared_sites_formated.csv')
labeled = pd.read_csv('measured_data.csv')
n_features = labeled.shape[1] - 2

In [4]:
# split into training and test sets
train_data2, test_data = train_test_split(labeled, test_size =.1, random_state = 1)
train_labels = train_data2['ZPPG_Performance']
train_data = train_data2.drop(['ZPPG_id', 'ZPPG_Performance'], axis=1)

print(f'train data: {len(train_data)}')
print(f'test data:  {len(test_data)}')

train data: 9366
test data:  1041


## utils

In [5]:
@torch.no_grad()
def evaluate(model, dataset):
  model.eval()
  val = dataset.drop(['ZPPG_id', 'ZPPG_Performance'], axis=1)
  preds = model(torch.tensor(val.values).float().to(device))
  labels = torch.tensor(dataset['ZPPG_Performance'].values).view(-1, 1).to(device)
  diff = preds.detach() - labels
  error = diff.abs().mean().item()
  model.train()
  return error

In [6]:
@torch.no_grad()
def topn(model, dataset, n=15):
  model.eval()
  to_drop = ['ZPPG_id']
  if 'ZPPG_Performance' in dataset: to_drop.append('ZPPG_Performance')
  preds = model(torch.tensor(dataset.drop(to_drop, axis=1).values).float().to(device))
  ids = dataset['ZPPG_id']
  preds_id = list(zip(
    preds.view(-1).tolist(),
    ids.values.tolist()))
  return sorted(preds_id, reverse=True)[:n]

## model and train

In [11]:
class MLP(nn.Module):
  def __init__(self, n_inputs=n_features, hidden=32, dropout=0.2):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(n_inputs, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, hidden),
        nn.BatchNorm1d(hidden),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(hidden, 1),
    )
  def forward(self, x):
    output = self.model(x)
    return output

In [12]:
model = MLP().to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.01)

In [19]:
def train(model, epochs=100000):
  model.train()
  labels = torch.tensor(labeled['ZPPG_Performance'].values).float().view(-1, 1).to(device)
  data = labeled.drop(['ZPPG_id', 'ZPPG_Performance'], axis=1)
  data = torch.tensor(data.values).float().to(device)
  # data = torch.tensor(train_data.values).float().to(device)
  # labels = torch.tensor(train_labels.values).float().view(-1, 1).to(device)
  for epoch in range(epochs):
    predictions = model(data)
    loss = F.mse_loss(predictions, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 200 == 0:
      error_test = error_train = evaluate(model, labeled)
      # error_test = evaluate(model, test_data)
      # error_train = evaluate(model, train_data2)
      print(f'{epoch=:5} {loss.item()=:7.4f} {error_test=:7.4f} {error_train=:7.4f}')
  model.eval()

train(model)

epoch=    0 loss.item()= 0.0011 error_test= 0.1141 error_train= 0.1141
epoch=  200 loss.item()= 0.0012 error_test= 0.1135 error_train= 0.1135
epoch=  400 loss.item()= 0.0011 error_test= 0.1139 error_train= 0.1139
epoch=  600 loss.item()= 0.0012 error_test= 0.1138 error_train= 0.1138
epoch=  800 loss.item()= 0.0011 error_test= 0.1128 error_train= 0.1128
epoch= 1000 loss.item()= 0.0012 error_test= 0.1144 error_train= 0.1144
epoch= 1200 loss.item()= 0.0012 error_test= 0.1143 error_train= 0.1143
epoch= 1400 loss.item()= 0.0011 error_test= 0.1140 error_train= 0.1140
epoch= 1600 loss.item()= 0.0012 error_test= 0.1145 error_train= 0.1145
epoch= 1800 loss.item()= 0.0012 error_test= 0.1124 error_train= 0.1124
epoch= 2000 loss.item()= 0.0011 error_test= 0.1133 error_train= 0.1133
epoch= 2200 loss.item()= 0.0012 error_test= 0.1144 error_train= 0.1144
epoch= 2400 loss.item()= 0.0012 error_test= 0.1139 error_train= 0.1139
epoch= 2600 loss.item()= 0.0012 error_test= 0.1141 error_train= 0.1141
epoch=

In [21]:
# save
# ----
# torch.save(model.state_dict(), 'weights/mlp.pt')

# load
# ----
# m = MLP().to(device)
# m.load_state_dict(torch.load('weights/mlp.pt'))

## eval

In [20]:
# predict solution
topn(model, unlabeled)

[(0.6385198831558228, 93762),
 (0.6374170184135437, 96286),
 (0.6371484398841858, 23565),
 (0.6263238191604614, 11558),
 (0.6240054368972778, 58945),
 (0.6227089166641235, 107278),
 (0.6225202083587646, 905),
 (0.6212724447250366, 88839),
 (0.620957612991333, 80395),
 (0.6194634437561035, 83512),
 (0.619322657585144, 94304),
 (0.6188409328460693, 92778),
 (0.6185967922210693, 94408),
 (0.6183627843856812, 62102),
 (0.6179928779602051, 105880)]

In [21]:
# validation
topn(model, test_data)
topn(model, train_data2)

[(1.8702857494354248, 3412),
 (1.8266371488571167, 2695),
 (1.8240541219711304, 7487),
 (1.8221070766448975, 2509),
 (1.821927547454834, 3773),
 (1.8169137239456177, 8721),
 (1.8135265111923218, 5386),
 (1.8075464963912964, 5751),
 (1.8061460256576538, 7243),
 (1.7997924089431763, 2004),
 (1.797755241394043, 5072),
 (1.7943958044052124, 4148),
 (1.7933684587478638, 6989),
 (1.791631817817688, 7146),
 (1.7850587368011475, 5667)]